# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

We'll be doing the following in today's notebook:

1. Task 1: Simple Assistant
2. Task 2: Adding Tools
  - Task 2a: Creating an Assistant with File Search Tool
  - Task 2b: Creating an Assistant with Code Interpreter Tool
  - Task 2c: Creating an Assistant with Function Calling Tool



---

Colab Specific Instructions:

To get started, please make a copy of this notebook using `File > Save a copy in Drive`

![image](https://i.imgur.com/rNzMEfs.png)

You will be expected to submit a GitHub link to the completed notebook, so if you're completing the assignment on Colab - you'll want to download the notebook when you have completed it.

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [44]:
!pip install -qU openai

In [45]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Task 1: Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [46]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [47]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Lola" # @param {type: "string"}
instructions = "You're a funny, charming, colorful assistant. " # @param {type: "string"}
model = "gpt-4o" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4", "gpt-4o"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [48]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [49]:
assistant

Assistant(id='asst_oJrTUoQL3oNfgj1tfxKG6yRn', created_at=1718139821, description=None, instructions="You're a funny, charming, colorful assistant. ", metadata={}, model='gpt-4o', name='Lola', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [50]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [51]:
thread

Thread(id='thread_VSMYW8ESCVJzoUachtD0jMem', created_at=1718139821, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [52]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How many Legos would it take to build the White House?"
)

Again, let's examine our `message` object!

In [53]:
message

Message(id='msg_ZhT11KXcTfgcxpYfW0ISryDd', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How many Legos would it take to build the White House?'), type='text')], created_at=1718139821, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VSMYW8ESCVJzoUachtD0jMem')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [54]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Please state the assumptions, state the methodology, and what questions would need to be answered. Then walk through the problem step-by and provide a numeric range at the end." # @param {type: "string"}

Let's run our Thread!

In [55]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions
)

Now that we've run our thread, let's look at the object!

In [56]:
run

Run(id='run_vFZUAbkiN4IOUQ1Tsh6rIJzn', assistant_id='asst_oJrTUoQL3oNfgj1tfxKG6yRn', cancelled_at=None, completed_at=None, created_at=1718139821, expires_at=1718140421, failed_at=None, incomplete_details=None, instructions='Please state the assumptions, state the methodology, and what questions would need to be answered. Then walk through the problem step-by and provide a numeric range at the end.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_VSMYW8ESCVJzoUachtD0jMem', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [57]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [58]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [59]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [60]:
messages.data[0]

Message(id='msg_FfK51D8mJzPS4HtzscTe19NB', assistant_id='asst_oJrTUoQL3oNfgj1tfxKG6yRn', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="To estimate the number of LEGO bricks required to build a model of the White House, we need to make several assumptions and outline a methodology. Then, we can calculate a numeric range for the number of pieces.\n\n### Assumptions:\n1. **Type of LEGO Bricks**: We'll assume a mixture of standard 2x4 and 1x2 LEGO bricks for simplicity.\n2. **Scale of the Model**: We'll use a specific scale to reduce the size of the LEGO model compared to the actual White House (e.g., 1:50 or 1:100).\n3. **Dimension Clarity**: We'll focus on the exterior dimensions of the White House main building.\n4. **Detail Level**: We assume a medium level of detail—enough to be recognizable but not replicating every minute feature.\n5. **Windows and Doors**: We'll assume some bricks will be substituted with transparent or specialized pi

### Streaming Our Runs

With recent upgrades to the Assistant API - we can now *stream* our outputs!

In order to do this - we'll need something called an `EventHandler` which will help us to decide on what actions to take based on the output of the LLM.

Let's build it below!

In [61]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

Now we can create our `run` and stream the output as it comes in!

In [62]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > To estimate the number of LEGO bricks required to build a model of the White House, we need to make several assumptions and outline a methodology. Then, we can calculate a numeric range for the number of pieces.

### Assumptions:
1. **Type of LEGO Bricks**: We'll assume a mixture of standard 2x4 and 1x2 LEGO bricks for simplicity.
2. **Scale of the Model**: We'll use a specific scale to reduce the size of the LEGO model compared to the actual White House (e.g., 1:50 or 1:100).
3. **Dimension Clarity**: We'll focus on the exterior dimensions of the White House main building.
4. **Detail Level**: We assume a medium level of detail—enough to be recognizable but not replicating every minute feature.
5. **Windows and Doors**: We'll assume some bricks will be substituted with transparent or specialized pieces for windows and doors but will approximate with standard bricks.

### Methodology:
1. **Determine the Scale**: Choose an appropriate scale for the LEGO model.
2. **Calculat

## Task 2: Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Task 2a: Creating an Assistant with the File Search Tool

The first thing we'll want to do is create an assistant with the File Search tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data to Vector Store

1. First, we need some data.
2. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [63]:
!wget https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt

--2024-06-11 14:04:18--  https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘paul_graham_essays.txt.2’

paul_graham_essays.     [ <=>                ] 266.67K  --.-KB/s    in 0.1s    

2024-06-11 14:04:19 (2.41 MB/s) - ‘paul_graham_essays.txt.2’ saved [273067]



Now we can upload our file to our Vector Store!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/file-search/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/file-search/vector-stores) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [64]:
vector_store = client.beta.vector_stores.create(name="Paul Graham Essay Compilation")

In [65]:
file_paths = ["paul_graham_essays.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

Let's look at what our `file_batch` contains!

In [66]:
while file_batch.status != "completed":
  time(1)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Your first GB is free and beyond that, usage is billed at $0.10/GB/day of vector storage. There are no other costs associated with vector store operations.

In [67]:
fs_assistant = client.beta.assistants.create(
  name=name,
  instructions=instructions,
  model=model,
  tools=[{"type": "file_search"}],
)

In [68]:
fs_assistant = client.beta.assistants.update(
  assistant_id=fs_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [69]:
fs_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What did Paul Graham say about Silicon Valley?",
    }
  ]
)

We can use an extension of the `EventHandler` that we created above to stream our `run`!

Let's add a few things:

1. A `on_tool_call_created` function which tells us which tool is being used.
2. A `on_message_done` that includes citations that were used by our File Search tool - this is like return the context *and* the response that we saw in our Pythonic RAG implementation.

In [70]:
class FSEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  @override
  def on_message_done(self, message) -> None:
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
      message_content.value = message_content.value.replace(
        annotation.text, f"[{index}]"
      )
      if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

Let's look at the final result!

In [71]:
with client.beta.threads.runs.stream(
  thread_id=fs_thread.id,
  assistant_id=fs_assistant.id,
  event_handler=FSEventHandler(),
) as stream:
  stream.until_done()


assistant > file_search


assistant > Paul Graham discusses Silicon Valley with a sense of admiration for its unique environment that fosters innovation and success. He highlights how Silicon Valley exemplifies the "right" conditions for startups due to the concentration of rich investors, which exceeds that of other regions significantly, thereby offering startup founders an optimal mix of resources and opportunities relatively unattained elsewhere[0] .
[0] paul_graham_essays.txt


### Task 2b: Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [72]:
ci_assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In [73]:
!git clone https://github.com/ali-ce/datasets.git

fatal: destination path 'datasets' already exists and is not an empty directory.


In [74]:
file = client.files.create(
  file=open("datasets/Y-Combinator/Startups.csv", "rb"),
  purpose='assistants'
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [75]:
ci_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "attachments": [
          {
              "file_id" : file.id,
              "tools" : [{"type" : "code_interpreter"}]
          }
      ]
    }
  ]
)

We'll once again need to create an `EventHandler`, except this time it will have an `on_tool_call_delta` method which will let us see the output of the code interpreter tool as well!

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [76]:
class CIEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

Once again, we can use the streaming interface thanks to creating our `EventHandler`!

In [77]:
with client.beta.threads.runs.stream(
  thread_id=ci_thread.id,
  assistant_id=ci_assistant.id,
  instructions=additional_instructions,
  event_handler=CIEventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import mimetypes

# define the file path
file_path = '/mnt/data/file-x1dP4U6wgH8NcQijkfcsqhHf'

# use mimetypes to guess the file type
file_type, _ = mimetypes.guess_type(file_path)

file_type
assistant > The `mimetypes` module was not able to determine the type of the file. Next, I will use other methods to inspect the file content to determine its type. This could involve reading its metadata or content. Let me read the initial bytes of the file and see if that gives any clues.# Open the file and read the first few bytes
with open(file_path, 'rb') as f:
    file_header = f.read(512)  # Read the first 512 bytes

file_header[:100]
assistant > Based on the initial bytes read from the file, it appears to be a CSV (Comma-Separated Values) file because its content is delimited by commas and follows a typical CSV structure.

The headers in the file appear to be:
- `Company`
- `Satus`
- `Year Founded`
- `Mapping Location`
- `Description`
- `Categories`
- `Found

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Task 2c: Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [78]:
!pip install -qU duckduckgo_search

In [79]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [80]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

"Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nThe Winnipeg Jets will have a captain for the 2023-24 season. ... Although it will be his first time serving as a team captain since his final year with the Swift Current Broncos in 2012-13, Lowry ...\nWinnipeg Jets. National team. Canada. NHL draft. 67th overall, 2011. Winnipeg Jets. Playing career. 2013-present. Adam Lowry (born March 29, 1993) is an American-born Canadian professional ice hockey centre and captain of the Winnipeg Jets of the National Hockey League (NHL).\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nWINNIPEG — The pride in Adam Lowry's voice was evident after being named captain of the Winnipeg Jets on Tuesday. Lowry

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [81]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

#### ❓ Question

Why does the description key-value pair matter?

#### 👩🏻‍💻 My Answer 👩🏻‍💻
In the context of the provided notebook segment, the description key-value pair within the `ddg_function` dictionary matters for several key reasons:

1. **Function Documentation**:
   - The `description` field helps document the purpose of the `duckduckgo_search` function. This is crucial for anyone reading the code to quickly understand what the function is designed to do, which is to "Answer non-technical questions."

2. **Clarity and Intention**:
   - The `description` provides a clear intention of the function's usage. By stating it answers non-technical questions, it sets the expectation for the type of queries this function is capable of handling.

3. **Guidance for API Users**:
   - When the function is part of an API, the `description` field guides the users (developers or other systems) on how to interact with the function. It informs them about the function's capabilities and appropriate use cases, ensuring they use it correctly.

4. **Enhancing User Experience**:
   - For applications or interfaces that use this function, displaying the description can help end-users understand what kind of questions they can ask. This improves user interaction and satisfaction by setting clear boundaries on the function’s scope.

5. **Contextual Search Queries**:
   - Within the parameters section, the `description` key under the `query` property provides an example of how to structure a search query. This is useful for both developers and automated systems to form correct and efficient search queries, improving the relevance and accuracy of search results.

By including the `description` key-value pair, the function becomes more user-friendly, easier to understand, and more efficient to use, both for developers and end-users. It adds an essential layer of context that aids in the proper utilization and integration of the function within larger systems or applications.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [82]:
fc_assistant = client.beta.assistants.create(
    name=name + " + Function Calling",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

Now we can create our thread, and attach our message to it - just as we've been doing!

In [83]:
fc_thread = client.beta.threads.create()
fc_message = client.beta.threads.messages.create(
  thread_id=fc_thread.id,
  role="user",
  content="Can you describe the Twitter beef between Elon and LeCun?",
)

Once again, we'll need an `EventHandler` for the streaming response - notice that this time we're utilizing a few new methods:

1. `handle_requires_action` - this will handle whatever action needs to take place in *our local environment*.
2. `submit_tool_outputs` - this will let us submit the resultant outputs from our local function call back to the Assistant run in the required format.

To be very clear and explicit - we'll be following this pattern:

1. Make a call to the LLM which will decide if a local function call is required.
2. If a local function call is required - send a response that indicates a local function call is required.
3. Call the function using the arguments provided by the LLM.
4. Return the response from the function call with LLM provided arguements.
5. Receive a response based on the output of the functional call from the LLM.

In [84]:
class FCEventHandler(AssistantEventHandler):
  @override
  def on_event(self, event):
    # Retrieve events that are denoted with 'requires_action'
    # since these will have our tool_calls
    if event.event == 'thread.run.requires_action':
      run_id = event.data.id  # Retrieve the run ID from the event data
      self.handle_requires_action(event.data, run_id)

  def handle_requires_action(self, data, run_id):
    tool_outputs = []

    for tool in data.required_action.submit_tool_outputs.tool_calls:
      print(tool.function.arguments)
      if tool.function.name == "duckduckgo_search":
        tool_outputs.append({"tool_call_id": tool.id, "output": duckduckgo_search(tool.function.arguments)})

    # Submit all tool_outputs at the same time
    self.submit_tool_outputs(tool_outputs, run_id)

  def submit_tool_outputs(self, tool_outputs, run_id):
    # Use the submit_tool_outputs_stream helper
    with client.beta.threads.runs.submit_tool_outputs_stream(
      thread_id=self.current_run.thread_id,
      run_id=self.current_run.id,
      tool_outputs=tool_outputs,
      event_handler=EventHandler(),
    ) as stream:
      for text in stream.text_deltas:
        print(text, end="", flush=True)
      print()

Thanks to our event handler - we can stream this process in our notebook!

In [85]:
with client.beta.threads.runs.stream(
  thread_id=fc_thread.id,
  assistant_id=fc_assistant.id,
  event_handler=FCEventHandler()
) as stream:
  stream.until_done()

{
  "query": "Twitter beef between Elon Musk and LeCun"
}

assistant > HoldHold onto onto your your feathers feathers,, because because the the Twitter Twitter beef beef between between Elon Elon Musk Musk and and Yann Yann Le LeCCunun is is hotter hotter than than a a jal jalapeapeñoño in in a a sauna sauna!

!

SoSo,, here's here's the the tea tea:: Elon Elon Musk Musk,, the the tech tech mog mogulul known known for for his his ventures ventures into into space space,, electric electric cars cars,, and and probably probably soon soon,, Mart Martianian real real estate estate,, had had a a bit bit of of a a t tiffiff with with Yann Yann Le LeCCunun,, the the noted noted AI AI researcher researcher and and Chief Chief AI AI Scientist Scientist at at Meta Meta ( (formerlyformerly Facebook Facebook).

).

ElElonon,, who's who's been been vocal vocal about about his his cautious cautious stance stance on on AI AI,, compared compared it it to to " "summsummoningoning the the demon demon"" 